# Markov Chains

Consider the following markov chains:

* An absorbing chain with states $\{1,2,3,4\}$. The transient states are $2$ and $3$, and the two absorbing barriers are states $1$ and $4$.
* An ergodic chain with 3 states

## Absorbic Markov Chain

Calculate the absorbic probabities of an absorbing markov chain and verify the result by simulation.

The transition probabilities for an example absorbic markov chain is shown in matrix $M_{abs}$.

$$M_{abs} = \begin{bmatrix} m_{11} & m_{12} & m_{13} & m_{14} \\ m_{21} & m_{22} & m_{23} & m_{24} \\ m_{31} & m_{32} & m_{33} & m_{34} \\ m_{41} & m_{42} & m_{43} & m_{44}\end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0.1 & 0.2 & 0.3 & 0.4 \\ 0.5 & 0.1 & 0.1 & 0.3 \\ 0 & 0 & 0 & 1\end{bmatrix}$$

In this example, state $1$ and $4$ are the absorbing barriers and state $2$ and $3$ are the transient states. The matrix of the transient states is $M_{\tau}$.

$$M_{\tau} = \begin{bmatrix} m_{22} & m_{23}\\ m_{32} & m_{33}\end{bmatrix} = \begin{bmatrix}0.2 & 0.3\\0.1 & 0.1\end{bmatrix}$$

The absorbing probabilities of this matrix are $f_{21}$, $f_{24}$, $f_{31}$, $f_{34}$. The probability of transitioning from state $2$ to end up in state $4$ is $f_{24}$, and the probability of transition from state $3$ to end up in state $4$ is $f_{34}$. The probability of transitioning from state $2$ to end up in state $1$ is $f_{21}$, and the probability of transition from state $3$ to end up in state $1$ is $f_{31}$. 

### Calculate Absorbic Probabilities

To calculate these absorbing probabilites with a closed form exact solution, the equation is as follows:
$$\begin{bmatrix} f_{24} \\ f_{34} \end{bmatrix} = (I-M_{\tau})^{-1}\begin{bmatrix} m_{24} \\ m_{34} \end{bmatrix} = \begin{bmatrix}1-0.2 & -0.3\\-0.1 & 1-0.1\end{bmatrix}^{-1}\begin{bmatrix} 0.4 \\ 0.3 \end{bmatrix}$$
$$\begin{bmatrix} f_{21} \\ f_{31} \end{bmatrix} = (I-M_{\tau})^{-1}\begin{bmatrix} m_{21} \\ m_{31} \end{bmatrix} = \begin{bmatrix}1-0.2 & -0.3\\-0.1 & 1-0.1\end{bmatrix}^{-1}\begin{bmatrix} 0.1 \\ 0.5 \end{bmatrix}$$

Also note that since the final state must be either $1$ or $4$ regardless of the starting state, $f_{21} + f_{24} = 1$, and $f_{31} + f_{34} = 1$.

These results will be calculated in the following code.

In [9]:
import numpy as np
from numpy.linalg import inv

In [32]:
# calculate absorbic properties
Mabs = np.matrix([[1,0,0,0], [0.1, 0.2, 0.3, 0.4], [0.5, 0.1, 0.1, 0.3], [0,0,0,1]])
Mtrans= Mabs[1:3, 1:3]
m4 = Mabs[1:3, 3] # the transition probabilities for state 4
m1 = Mabs[1:3, 0] # the transition probabilities for state 1

# absorbing probabilities for state 4:
f4 = np.dot(inv(np.identity(2)- Mtrans),m4)

# absorbing probabilities for state 4:
f1 = np.dot(inv(np.identity(2)- Mtrans),m1)

matrix([[ 1.],
        [ 1.]])

In [34]:
# confirm that f1+f4=1
f1+f4

matrix([[ 1.],
        [ 1.]])

In [149]:
fprobs = np.concatenate((f1, f4), axis=1)

matrix([[ 0.34782609,  0.65217391],
        [ 0.5942029 ,  0.4057971 ]])

### Verify Results by Simulation

Since every starting state will eventually get stuck in either state $1$ or state $4$, run a simulation of the markov chain until one of these two states is reached. Repeat this for many experiments, to determine the absorbic probabilities of each absorbing state.

To start the simulation, the distribution for the starting state is required: $\Pi (0)$.

This can be specified by a vector of probabilities of starting in each of the 4 states. Once the starting state is chosen, it would be denoted by a 1 in that position of a vector and 0 in all other positions. 

To determine the state after one step in the chain, $\Pi(1)$, this would be determined by the state at time $0$, and the transition probabilities of that state. For example, if the state chosen for $\Pi (0)$ is $2$, then the next state chosen will be determined by the transition probablities associated with state $2$.

$$\Pi(1) = M_{abs}^T\Pi(0) = M{abs}^T\begin{bmatrix} 0 \\ 1 \\ 0 \\ 0\end{bmatrix}$$

In [106]:
# choose starting state:
state0 = 2

# create the state vector
statevec = np.zeros((numstates,1))
statevec[state0-1]=1

# get transition probabilities of current state
statetransprobs = np.dot(Mabs.T, statevec)

# choose the next state based on the current state transition probabilities
np.random.seed(7)
currstate = np.random.choice(a= list(range(numstates)), p=np.squeeze(np.asarray(statetransprobs)))+1

# this completes one step of the markov chain, repeat until it reaches either state 1 or 4
currstate

1

In [114]:
# iterate through markov chain until it reaches an absorbic state
absorbic_states = [1, 4]
currstate=2
np.random.seed(100)

while currstate not in absorbic_states:
    statevec = np.zeros((numstates,1))
    statevec[currstate-1]=1
    statetransprobs = np.dot(Mabs.T, statevec)
    print(currstate)
    currstate = np.random.choice(a= list(range(numstates)), p=np.squeeze(np.asarray(statetransprobs)))+1
    
print("The final state is: " + str(currstate))

2
3
The final state is: 1


The previous simulation shows the result of running the markov chain once. Due to random probability of choosing states, the results will be different from running it several times, so to get the absorption probabilities for state $1$ and $4$ from starting from state $2$, run this chain many times, and count the number of times in each final state to get the distribution.

In [160]:
# run markov chain many times to compute distribution
np.random.seed(77)
absorbic_states = [1, 4]
transient_states = [2, 3]
numexp=10000
simfprobs = np.zeros((2,2))

for transstate in transient_states:
    finalstates = np.zeros(numexp)
    for i in range(numexp):
        currstate=transstate
        while currstate not in absorbic_states:
            statevec = np.zeros((numstates,1))
            statevec[currstate-1]=1
            statetransprobs = np.dot(Mabs.T, statevec)
            currstate = np.random.choice(a= list(range(numstates)), p=np.squeeze(np.asarray(statetransprobs)))+1
        finalstates[i]=currstate
    unique, counts = np.unique(finalstates, return_counts=True)
    #finalstatecounts= dict(zip(unique, counts)) 
    simfprobs[transstate-2,:]=counts/numexp
    


In [162]:
print("from simulation:")
print("The final state absorbing probabilities starting from state 2 are, f21: " + str(simfprobs[0,0])+ " and f24: "+str(simfprobs[0,1]))
print("The final state absorbing probabilities starting from state 3 are, f31: " + str(simfprobs[1,0])+ " and f34: "+str(simfprobs[1,1]))

print("from exact calculation:")
print("The final state absorbing probabilities starting from state 2 are, f21: " + str(fprobs[0,0])+ " and f24: "+str(fprobs[0,1]))
print("The final state absorbing probabilities starting from state 3 are, f31: " + str(fprobs[1,0])+ " and f34: "+str(fprobs[1,1]))

print("Difference between simulation and calculation:")
print("f21: " + str(simfprobs[0,0]-fprobs[0,0]))
print("f24: " + str(simfprobs[0,1]-fprobs[0,1]))
print("f21: " + str(simfprobs[1,0]-fprobs[1,0]))
print("f21: " + str(simfprobs[,0]-fprobs[0,0]))

from simulation:
The final state absorbing probabilities starting from state 2 are, f21: 0.3514 and f24: 0.6486
The final state absorbing probabilities starting from state 3 are, f31: 0.5954 and f34: 0.4046
from exact calculation:
The final state absorbing probabilities starting from state 2 are, f21: 0.348 and f24: 0.652
The final state absorbing probabilities starting from state 3 are, f31: 0.611 and f34: 0.389
Difference between simulation and calculation:
f21: 0.0034
